In [1]:
import fastf1
from fastf1 import plotting
import pandas as pd
from data_schema import RacePredictionFeatures, PreraceFeatures, QualifyingFeatures, PastLapsFeatures, TelemetryFeatures
from datetime import datetime


In [2]:
qsession = fastf1.get_session(2024, 'Monaco', 'Q')
qsession.load()

req         WARNING 	DEFAULT CACHE ENABLED! (178.94 MB) C:\Users\Bibidh Subedi\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Monaco Grand Prix - Qualifying [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '31', '3', '18', '27', '14', '2', '20', '11', '77', '24']


In [3]:
session = fastf1.get_session(2024, 'Monaco', 'R')
session.load()

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']


In [4]:
drivers = ["VER","NOR"]

In [5]:
driver = drivers[1]
laps = session.laps.pick_drivers(driver)
keys = session.car_data.keys()
car_data = session.car_data['4']
pos_data = session.pos_data['4']

qlaps = qsession.laps.pick_drivers(driver)
qcar_data = qsession.car_data['4']
qpos_data = qsession.pos_data['4']

# PRERACE FEATURES
This needs to be from some external sources i guess? or idk basically definately not a part of the the session

In [6]:
session.weather_data

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:54.094000,21.1,69.0,1019.1,False,49.4,27,1.1
1,0 days 00:01:54.088000,21.1,68.0,1019.1,False,49.5,2,0.8
2,0 days 00:02:54.095000,21.0,68.0,1019.2,False,50.0,3,0.9
3,0 days 00:03:54.083000,21.0,68.0,1019.1,False,48.3,8,0.7
4,0 days 00:04:54.095000,20.9,66.0,1019.1,False,48.4,107,1.2
...,...,...,...,...,...,...,...,...
195,0 days 03:15:54.885000,21.7,61.0,1018.4,False,40.6,224,0.9
196,0 days 03:16:54.899000,21.8,61.0,1018.4,False,40.9,219,1.2
197,0 days 03:17:54.901000,21.8,61.0,1018.4,False,40.9,284,0.8
198,0 days 03:18:54.896000,21.8,62.0,1018.4,False,40.2,0,0.8


In [7]:
prerace = {
    'driverChampPoints': 275,      # Placeholder
    'constructorChampPoints': 400, # Placeholder
    'trackType': 'street',
    'expectedWeather': 'mixed'
}
prerace

{'driverChampPoints': 275,
 'constructorChampPoints': 400,
 'trackType': 'street',
 'expectedWeather': 'mixed'}

# QUALIFYING FEATURES

In [8]:
qbest3_laps = qlaps.nsmallest(3, 'LapTime')
qbest3_avg = qbest3_laps['LapTime'].mean().total_seconds() if not qbest3_laps.empty else None
fastest_lap = qlaps.pick_fastest()
pole_time = qsession.laps.pick_fastest()['LapTime'].total_seconds()
grid_position = qsession.results.loc[driver, 'GridPosition'] if driver in qsession.results.index else None

qualifying = {
    'gridPosition': grid_position,
    'best3LapAvg': qbest3_avg,
    'deltaToPole': fastest_lap['LapTime'].total_seconds() - pole_time if fastest_lap is not None else None,
    'tireBestLap': fastest_lap['Compound'] if fastest_lap is not None else None
}

qualifying

{'gridPosition': None,
 'best3LapAvg': 70.738,
 'deltaToPole': 0.27200000000000557,
 'tireBestLap': 'SOFT'}

# PAST LAPS FEATURES

In [9]:
current_lap = laps.iloc[-1]['LapNumber'] if not laps.empty else None
position = laps.iloc[-1]['Position'] if not laps.empty else None
recent_laps = laps.tail(5)
avg_lap_time = recent_laps['LapTime'].mean().total_seconds() if not recent_laps.empty else None
# lap_time_var = recent_laps['LapTime'].var() if not recent_laps.empty else None?
num_pit_stops = laps['PitInTime'].count()
positions_gained_lost = grid_position - position if grid_position and position else None

past_laps = {
    'currentLap': current_lap,
    'position': position,
    'avgLapTime': avg_lap_time,
    'numPitStops': num_pit_stops,
    'timeLostInPits': None,  # Needs calculation
    'positionsGainedLost': positions_gained_lost,
    'deltaToLeader': None,   # Needs calculation
    'gapAhead': None,        # Needs calculation
    'gapBehind': None        # Needs calculation
}

past_laps

{'currentLap': np.float64(78.0),
 'position': np.float64(4.0),
 'avgLapTime': 76.685,
 'numPitStops': np.int64(1),
 'timeLostInPits': None,
 'positionsGainedLost': None,
 'deltaToLeader': None,
 'gapAhead': None,
 'gapBehind': None}

In [10]:
avg_speed = car_data['Speed'].mean() if not car_data.empty else None
throttle_avg = car_data['Throttle'].mean() if 'Throttle' in car_data.columns else None
throttle_var = car_data['Throttle'].var() if 'Throttle' in car_data.columns else None
brake_usage_pct = (car_data['Brake'] > 0).mean() * 100 if 'Brake' in car_data.columns else None
drs_usage_freq = (car_data['DRS'] > 0).sum() if 'DRS' in car_data.columns else None
gear_shifts_per_lap = car_data['nGear'].diff().abs().sum() if 'nGear' in car_data.columns else None
steering_var = car_data['Steer'].var() if 'Steer' in car_data.columns else None

telemetry = {
    'avgSpeed': avg_speed,
    'throttleAvg': throttle_avg,
    'throttleVar': throttle_var,
    'brakeUsagePct': brake_usage_pct,
    'drsUsageFreq': drs_usage_freq,
    'gearShiftsPerLap': gear_shifts_per_lap,
    'steeringVar': steering_var,
    'relPosX': pos_data['X'].iloc[-1] if not pos_data.empty else None,
    'relPosY': pos_data['Y'].iloc[-1] if not pos_data.empty else None,
    'relPosZ': pos_data['Z'].iloc[-1] if not pos_data.empty else None,
    'sectorMicroDeltas': None # Needs custom calculation
}

In [11]:
example = RacePredictionFeatures(
    driverId="max_verstappen",
    teamId="red_bull_racing",
    timestamp=pd.Timestamp.now().to_pydatetime(),
    prerace=PreraceFeatures(**prerace),
    qualifying=QualifyingFeatures(**qualifying),
    pastLaps=PastLapsFeatures(**past_laps),
    telemetry=TelemetryFeatures(**telemetry)
)

In [12]:
example

RacePredictionFeatures(driverId='max_verstappen', teamId='red_bull_racing', timestamp=datetime.datetime(2025, 8, 8, 22, 43, 49, 296624), prerace=PreraceFeatures(driverChampPoints=275, constructorChampPoints=400, trackType='street', expectedWeather='mixed'), qualifying=QualifyingFeatures(gridPosition=None, best3LapAvg=70.738, deltaToPole=0.27200000000000557, tireBestLap='SOFT'), pastLaps=PastLapsFeatures(currentLap=78, position=4, avgLapTime=76.685, numPitStops=1, timeLostInPits=None, positionsGainedLost=None, deltaToLeader=None, gapAhead=None, gapBehind=None), telemetry=TelemetryFeatures(avgSpeed=81.56585968161573, throttleAvg=29.70337787185254, brakeUsagePct=21.718040305155004, drsUsageFreq=22581.0, gearShiftsPerLap=3238))

In [13]:
json_data = example.model_dump_json(indent=4)
print(json_data)

{
    "driverId": "max_verstappen",
    "teamId": "red_bull_racing",
    "timestamp": "2025-08-08T22:43:49.296624",
    "prerace": {
        "driverChampPoints": 275,
        "constructorChampPoints": 400,
        "trackType": "street",
        "expectedWeather": "mixed"
    },
    "qualifying": {
        "gridPosition": null,
        "best3LapAvg": 70.738,
        "deltaToPole": 0.27200000000000557,
        "tireBestLap": "SOFT"
    },
    "pastLaps": {
        "currentLap": 78,
        "position": 4,
        "avgLapTime": 76.685,
        "numPitStops": 1,
        "timeLostInPits": null,
        "positionsGainedLost": null,
        "deltaToLeader": null,
        "gapAhead": null,
        "gapBehind": null
    },
    "telemetry": {
        "avgSpeed": 81.56585968161573,
        "throttleAvg": 29.70337787185254,
        "brakeUsagePct": 21.718040305155004,
        "drsUsageFreq": 22581.0,
        "gearShiftsPerLap": 3238
    }
}


In [14]:
grid_positions = qsession.results
for i in range(20):
    if grid_positions[i:i+1]["DriverNumber"] == 4:
        driver_position = qsession.results[qsession.results['Driver'] == 4]['Position'].values[0]
        break


# driver_position = 0

# for i in range(20):
#     if grid_positions["DriverNumber"] == 4:
#         driver_position = qsession.results[qsession.results['Driver'] == 4]['Position'].values[0]

# driver_position
# driver_position = qsession.results[qsession.results['Driver'] == 4]['Position'].values[0]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().